In [6]:
from typing import Callable, Set, Generator, Iterable, Dict
import datetime
import collections
import functools

import pymysql
import helpers

In [7]:
db_host = 'localhost'
db_database = 'boe'
db_user = 'root'
db_password = 'pass'

use_db_settings = lambda func, *args: func(db_host, db_user, db_password, db_database, *args)

# Boe database initial setup

Supported character sets and collations in __MariaDB__:
https://mariadb.com/kb/en/supported-character-sets-and-collations/

### PyMysql utilities

In [8]:
def object_exists(cursor: pymysql.cursors.Cursor, query: str, object_name: str) -> bool:
    '''Check if an SQL object is present in the server.'''
    
    cursor.execute(query)
    objects = cursor.fetchall()
    exists = helpers.pipe(objects,
        functools.partial(map, lambda x: x[0]),
        lambda x: object_name in x)
    
    return exists

def database_exists(cursor: pymysql.cursors.Cursor, db_name:str) -> bool:
    '''Check if the database is present in the SQL sever.'''
    
    return object_exists(cursor, 'show databases', db_name)

def table_exists(cursor: pymysql.cursors.Cursor, table_name:str) -> bool:
    '''Check if the table is present in the given database.'''
    
    return object_exists(cursor, 'show tables', table_name)

def create_database(db_host: str, db_user: str, db_password: str, db_database: str, force=False) -> None:
    '''Create the given database.'''
    
    connection = pymysql.connect(host=db_host,
                                 user=db_user,
                                 password=db_password)

    with connection:
        cursor = connection.cursor()
        if not database_exists(cursor, db_database):
            cursor.execute(f'create database {db_database}')
        elif force is True:
            cursor.execute(f'drop database {db_database}')
            cursor.execute(f'create database {db_database}')
            
def create_tables(use_db_settings: Callable, *tables) -> None:
    '''Create the given tables provided as name, query arguments.
    >>> create_tables(db_host, 
              db_user, 
              db_password,
              db_database,
              'boe_diary_section',
              boe_diary_section_table,
              'boe_diary_entry',
              boe_diary_entry_table)
    '''
    
    connection = use_db_settings(pymysql.connect)

    with connection:
        cursor = connection.cursor()
        for table_name, table_query in zip(tables[::2], tables[1::2]):
            if not table_exists(cursor, table_name):
                cursor.execute(table_query)

### Database setup utilities

In [20]:
boe_sections = {
    '1': 'disposiciones generales',
    '2': 'autoridades y personal',
    '2a': 'nombramientos situaciones e incidencias',
    '2b': 'oposiciones y concursos',
    '3': 'otras secciones',
    '4': 'administración de justicia',
    '5': 'anuncios',
    '5a': 'licitaciones públicas y adjudicaciones',
    '5b': 'otros anuncios particulares',
    '5c': 'anuncios particulares',
    't': 'tribunal constitucional'
}

boe_diary_section_table = '''
CREATE TABLE boe_diary_section (
    id VARCHAR(2) NOT NULL,
    name VARCHAR(60) NOT NULL,
    PRIMARY KEY (id)
) character set=utf8;
'''

boe_diary_entry_label_table = '''
CREATE TABLE boe_diary_entry_label (
    name VARCHAR(500),
    entry VARCHAR(20) NOT NULL,
    FOREIGN KEY (entry)
        REFERENCES boe_diary_entry (id)
        ON DELETE CASCADE
) character set=utf8;
'''

boe_diary_entry_references_table = '''
CREATE TABLE boe_diary_entry_reference (
    source VARCHAR(20) NOT NULL,
    referenced VARCHAR(20) NOT NULL,
    FOREIGN KEY (source)
        REFERENCES boe_diary_entry (id)
        ON DELETE CASCADE,
    FOREIGN KEY (referenced)
        REFERENCES boe_diary_entry (id)
        ON DELETE CASCADE
) character set=utf8;
'''

boe_diary_entry_table = '''
CREATE TABLE boe_diary_entry (
    id VARCHAR(20) NOT NULL,
    date DATE,
    title VARCHAR(1000),
    section VARCHAR(2) NOT NULL,
    department VARCHAR(500),
    epigraph VARCHAR(1000),
    pdf_url VARCHAR(240),
    xml_url VARCHAR(240),
    htm_url VARCHAR(240),
    has_economic_impact VARCHAR(20),
    PRIMARY KEY (id),
    FOREIGN KEY (section)
        REFERENCES boe_diary_section (id)
        ON DELETE CASCADE
) character set=utf8;
'''

In [10]:
def insert_sections(use_db_settings: Callable, sections: Dict) -> None:
    '''Insert the information for the different BOE sections.'''
    
    connection = use_db_settings(pymysql.connect)

    with connection:
        cursor = connection.cursor()
        if table_exists(cursor, 'boe_diary_section'):
            for section_id, section_name in boe_sections.items():
                query = f"INSERT INTO boe_diary_section VALUES ('{section_id}', '{section_name}')"
                cursor.execute(query)

### Database setup execution

In [5]:
use_db_settings(create_database)

create_tables(use_db_settings,
              'boe_diary_section',
              boe_diary_section_table,
              'boe_diary_entry',
              boe_diary_entry_table,
              'boe_diary_label',
              boe_diary_label_table,
              'boe_diary_entry_references',
              boe_diary_entry_references_table)

insert_sections(use_db_settings, boe_sections)

NameError: name 'create_database' is not defined

In [23]:
create_tables(use_db_settings,
              'boe_diary_entry_label',
              boe_diary_entry_label_table,)